In [6]:
import pygetwindow as gw

def get_active_window(target_window):
    active_window = gw.getActiveWindow()
    return (active_window and target_window == active_window.title)

In [7]:
import ctypes
import time
import datetime

def listen_loading():
    # Define necessary structures and constants
    class CURSORINFO(ctypes.Structure):
        _fields_ = [("cbSize", ctypes.c_uint),
                    ("flags", ctypes.c_uint),
                    ("hCursor", ctypes.c_void_p),
                    ("ptScreenPos", ctypes.wintypes.POINT)]

    # Cursor types for default and loading (hourglass/spinning wheel)
    IDC_ARROW = 32512
    IDC_WAIT = 32514

    # Load default and loading cursors
    user32 = ctypes.WinDLL('user32')
    LoadCursor = user32.LoadCursorW
    GetCursorInfo = user32.GetCursorInfo

    # Function to load specific cursors
    def load_cursor(cursor_id):
        return LoadCursor(None, cursor_id)

    # Default and loading cursor handles
    default_cursor = load_cursor(IDC_ARROW)
    loading_cursor = load_cursor(IDC_WAIT)

    def check_cursor():
        cursor_info = CURSORINFO()
        cursor_info.cbSize = ctypes.sizeof(CURSORINFO)
        
        if GetCursorInfo(ctypes.byref(cursor_info)):
            return cursor_info.hCursor
        

    previous_cursor = None

    while True:
        current_cursor = check_cursor()

        if current_cursor == default_cursor and previous_cursor == "Loading":
            timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]  # Format with milliseconds
            return timestamp
        
        elif current_cursor == loading_cursor:
            previous_cursor = "Loading"

        time.sleep(0.1)  # Adjust the interval to 0.1 seconds

In [11]:
import win32api
import time


def intialise_on_click(target_window): 
    state_left = win32api.GetKeyState(0x01)  # Left button up = 0 or 1. Button down = -127 or -128

    started = False

    while True:
        a = win32api.GetKeyState(0x01)
        if a != state_left:  # Button state changed
            state_left = a
            if a < 0 and get_active_window(target_window) and not started:
                print(f"Window '{target_window}' gained focus at {time.strftime('%Y-%m-%d %H:%M:%S')}")
                monitoring_time = listen_loading()
                print(f"Started Logging at {monitoring_time}")
                started = True
            
            # Assume that if already focused and already started, means you are stopping (AND NOT CLICKING ANYTHING ELSE)
            elif a < 0 and get_active_window(target_window) and started:
                timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]  # Format with milliseconds
                print(f"Stopped Logging at {timestamp}")
                break
                
        time.sleep(0.001)

intialise_on_click("Maxim DeviceStudio - [PPG EV Kit]")

Window 'Maxim DeviceStudio - [PPG EV Kit]' gained focus at 2024-09-02 18:06:21
2024-09-02 18:06:30.982
done


In [35]:
import csv

def convert_to_seconds(time_details):
    # Expecting HH:MM:SS.T format
    hour, minute, seconds = time_details.split(":")
    full_timing = int(hour)*60*60 + int(minute)*60 + float(seconds)
    return full_timing


def convert_csv_timestamps(input_file, output_file):
    with open(input_file, mode='r') as infile, open(output_file, mode='w', newline='') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)

        #read the header row and write it directly to the output file
        header = next(reader)
        writer.writerow(header)

        #base header
        base_row = next(reader)
        full_timestamp = base_row[0]
        split_timestamp = full_timestamp.split(" ")
        base_timing = convert_to_seconds(split_timestamp[-1])
        base_row[0] = 0
        writer.writerow(base_row)

        for row in reader:
            split_timestamp = row[0].split(" ")
            cell_timing = convert_to_seconds(split_timestamp[-1])
            time_interval = cell_timing - base_timing
            row[0] = time_interval
            writer.writerow(row)

In [38]:
convert_csv_timestamps('PPG_2024-09-02_17-32-33.csv', 'output2.csv')
